# Libraries

In [4]:
import numpy as np
import pandas as pd
import os

# Data load

In [3]:
file_names = os.listdir("./data")
files_text = []

for file_name in file_names:
    with open(f"./data/{file_name}", "r") as f:
        files_text.append(f.readline())

# Frequency of letters

In [17]:
frequency = pd.DataFrame()

for file_text, file_name in zip(files_text, file_names):
    signs, counts = np.unique(np.array(list(files_text[0])), return_counts=True)
    column = pd.DataFrame(counts, index=signs)
    frequency = pd.concat([frequency, column], axis=1)

frequency.columns = file_names

In [18]:
frequency

,norm_hamlet.txt,norm_romeo_and_juliet.txt,norm_wiki_sample.txt
,31819,31819,31819
a,9521,9521,9521
b,1795,1795,1795
c,2624,2624,2624
d,5060,5060,5060
e,14797,14797,14797
f,2722,2722,2722
g,2268,2268,2268
h,8164,8164,8164
i,8396,8396,8396


# First-order approximation

# Conditional probability of letters

# Approximations based on Markov sources